# IMPORTING ALL LIBRARIES

In [63]:
import numpy
import tensorflow as tf
import pandas
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.utils import shuffle

#  fix random seed for reproducibility

In [64]:
seed = 7
numpy.random.seed(seed)

# load dataset

In [65]:
dataframe = pandas.read_csv('sonar.csv', header= None)
dataset = dataframe.values
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

X[0].shape

(60,)

In [66]:
    encoder_Y = preprocessing.LabelEncoder()
    encoder_Y.fit(Y)
LabelEncoder()

LabelEncoder()

In [67]:
encoder_Y.classes_

array(['M', 'R'], dtype=object)

In [68]:
encoder_Y.transform(Y)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# baseline model

In [69]:

# baseline model
def create_baseline():
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model, write code below
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

# evaluate model with standardized dataset

In [70]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Results: 86.49% (4.82%)


# evaluate baseline model with standardized dataset

In [71]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 84.59% (3.67%)


# smaller model

In [72]:
def create_smaller():
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model, write code below
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [73]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 84.57% (5.24%)


# larger model

In [74]:
def larger_model():
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [75]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=larger_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 83.64% (6.83%)


# functional model

In [76]:
def functional_model():
    inputs = keras.Input(shape=(60,))
    layer1 = Dense(60, activation='relu')(inputs)
    layer2 = Dense(60, activation='relu')(layer1)
    outputs = Dense(1, activation='sigmoid')(layer2)

    model = keras.Model(inputs, outputs)
    
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [77]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=functional_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Functional Api: Accuracy %.2f%%, Error (%.2f%%)" % (results.mean()*100, results.std()*100))

Functional Api: Accuracy 83.09%, Error (4.69%)


# Rewriting the code using the Keras Functional API

In [78]:
from keras.layers import Input
from keras.layers import Dense
from keras.utils import plot_model
from keras.models import Model

Inputs = Input(shape=(60,))
x = Dense(60, activation='relu')(Inputs)
x = Dense(60, activation='relu')(x)
x = Dense(60, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=Inputs, outputs=output)

print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        (None, 60)                0         
_________________________________________________________________
dense_240 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_241 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_242 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_243 (Dense)            (None, 1)                 61        
Total params: 11,041
Trainable params: 11,041
Non-trainable params: 0
_________________________________________________________________
None


# Model Subclassing 

In [79]:
import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = Dense(60, activation = 'relu')
        self.dense2 = Dense(60, activation = 'relu')
        self.dense3 = Dense(1, activation = 'sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)
    
model = MyModel()

# Rewriting the code without using scikit-learn

In [80]:
dataframe = shuffle(dataframe)

dataset = dataframe.values

X = dataset[:, 0:60].astype(float)
y = dataset[:, 60]

# label encoding
le = LabelEncoder()
encoded_y = le.fit(y).transform(y)

k=10
num_val_samples = len(X)//k
num_epochs = 100

all_acc = numpy.array([])
# all_err = np.array([])

In [81]:
for i in range(k):
    print('Processing K-Fold #', i+1)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_y[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = numpy.concatenate(
        [X[:i * num_val_samples],
         X[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = numpy.concatenate(
        [encoded_y[:i * num_val_samples],
         encoded_y[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = functional_model()
    history = model.fit(partial_train_data, partial_train_targets,
        epochs=num_epochs, batch_size=5, verbose=0, validation_data=(val_data, val_targets))
    
    results = model.evaluate(val_data, val_targets, verbose=0)
    
#     all_err = np.append(all_err, results[0])
    all_acc = numpy.append(all_acc, results[1])
    
print("+--------------------+")    
print("Custom K-Folds: Accuracy %.2f%%, Error (%.2f%%)" % (all_acc.mean()*100, all_acc.std()*100))

Processing K-Fold # 1
Processing K-Fold # 2
Processing K-Fold # 3
Processing K-Fold # 4
Processing K-Fold # 5
Processing K-Fold # 6
Processing K-Fold # 7
Processing K-Fold # 8
Processing K-Fold # 9
Processing K-Fold # 10
+--------------------+
Custom K-Folds: Accuracy 83.00%, Error (7.48%)
